### some variables

In [1]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = False
model_checkpoint = "bert-base-uncased"
batch_size = 5
num_train_epochs = 1

In [2]:
max_length = 384 # this refers to max number of TOKENS - not characters
doc_stride = 128 # this is number of overlap, so we do not split long documents inside an answer

In [25]:
output_model_name = model_checkpoint + " finetuned"

### 1. Load squad data

In [5]:
from datasets import load_dataset, load_metric, load_from_disk
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import utilities as u

In [6]:
path = r"C:\Users\tanch\Documents\GitHub\Covid-19-QA-System\URECA Research\SQUAD format data\SQUAD subset seed 0"
datasets = load_from_disk(path)
datasets

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 5000
    })
})

### 2. Prepare Training Samples

In [8]:
# instantiate the tokenzier 
# note that different models require different tokenizers
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
# check that the tokenizer we instantiated  is a fast tokenizer because we need its special features
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

- these models have fast tokenizers
    - https://huggingface.co/transformers/index.html#bigtable

In [10]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    pad_on_right = tokenizer.padding_side == "right"
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [11]:
# more samples have been produced due the the splitting function
# transformers uses smart caching - the following code needs to be run only once as subsequent runs uses cached data
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 20206
    })
    validation: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions', 'token_type_ids'],
        num_rows: 5106
    })
})

### 3. Load pretrained model

In [13]:
# import mlflow
# mlflow.end_run()                         # ends previous run

In [12]:
# Initialise the QA model
# the warning is telling us that we need to fine tune the QA model - it has not been fine tuned and cannot do any QA problems
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

### 4. TrainingArguments"
- required to customise our "Trainer" to do the training


In [13]:
args = TrainingArguments(
    f"test-squad",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
)

### 5. default_data_collator
- batches our processed training examples together

In [14]:
from transformers import default_data_collator
data_collator = default_data_collator

### 6. use GPU

In [15]:
import torch
import gc

In [16]:
gc.collect()
torch.cuda.empty_cache()                   # empty cache so cude can be used 

In [18]:
model.device                         # device using cpu

device(type='cpu')

In [19]:
model.to(torch.device('cuda:0'))    # tells model to use cude number 0

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [20]:
model.device                         # device using cuda

device(type='cuda', index=0)

### 7. Trainer 
- performs the training/fine tuning of BERT

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [22]:
# .train() begins training
%time trainer.train()

2021/06/14 17:38:21 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Step,Training Loss
500,2.900472
1000,1.806506
1500,1.554333
2000,1.471918
2500,1.485569
3000,1.374193
3500,1.373221
4000,1.355788


Wall time: 38min 39s


TrainOutput(global_step=4042, training_loss=1.6612076237908335)

### 8. Save model

In [26]:
model.save_pretrained(save_directory = r"C:\Users\tanch\Documents\GitHub\Covid-19-QA-System\URECA Research\models"+f"\\{output_model_name}", )